In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/rushing/season/2020/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Rushing Attempts', 'Rushing Yards',
                 'Avg Yards per Rush Attempt', 'Long Rushing', '20+ Yard Rushes', 'Rushign Touchdowns',
                 'Rushing Yards per Game', 'Rushing Fumbles', 'Rushing Fumbles Lost', 'Rushing First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs
0,RB,16,378,"2,027",5.4,94,16,17,126.7,3,2,98
1,RB,14,312,"1,557",5.0,70,6,16,111.2,4,2,91
2,RB,15,232,"1,169",5.0,62,7,11,77.9,1,1,69
3,RB,14,201,"1,104",5.5,77,6,9,78.9,1,0,52
4,RB,15,247,"1,070",4.3,80,5,8,71.3,0,0,59
...,...,...,...,...,...,...,...,...,...,...,...,...
367,QB,16,18,-8,-0.4,3,0,0,-0.5,0,0,0
368,QB,8,13,-9,-0.7,0,0,0,-1.1,0,0,0
369,WR,16,3,-12,-4.0,-1,0,0,-0.8,0,0,0
370,RB,1,1,-12,-12.0,-12,0,0,-12.0,0,0,0


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2020
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Derrick Henry,TEN,RB,16,378,"2,027",5.4,94,16,17,126.7,3,2,98,2020
1,2,Dalvin Cook,MIN,RB,14,312,"1,557",5.0,70,6,16,111.2,4,2,91,2020
2,3,Jonathan Taylor,IND,RB,15,232,"1,169",5.0,62,7,11,77.9,1,1,69,2020
3,4,Aaron Jones,GB,RB,14,201,"1,104",5.5,77,6,9,78.9,1,0,52,2020
4,5,David Montgomery,CHI,RB,15,247,"1,070",4.3,80,5,8,71.3,0,0,59,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,368,Philip Rivers,IND,QB,16,18,-8,-0.4,3,0,0,-0.5,0,0,0,2020
368,369,Tim Boyle,GB,QB,8,13,-9,-0.7,0,0,0,-1.1,0,0,0,2020
369,370,Cedrick Wilson Jr.,DAL,WR,16,3,-12,-4.0,-1,0,0,-0.8,0,0,0,2020
370,370,Adrian Killins,PHI,RB,1,1,-12,-12.0,-12,0,0,-12.0,0,0,0,2020


In [6]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Derrick Henry,TEN,RB,16,378,"2,027",5.4,94,16,17,126.7,3,2,98,2020
1,2,Dalvin Cook,MIN,RB,14,312,"1,557",5.0,70,6,16,111.2,4,2,91,2020
2,3,Jonathan Taylor,IND,RB,15,232,"1,169",5.0,62,7,11,77.9,1,1,69,2020
3,4,Aaron Jones,GB,RB,14,201,"1,104",5.5,77,6,9,78.9,1,0,52,2020
4,5,David Montgomery,CHI,RB,15,247,"1,070",4.3,80,5,8,71.3,0,0,59,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,368,Philip Rivers,IND,QB,16,18,-8,-0.4,3,0,0,-0.5,0,0,0,2020
368,369,Tim Boyle,GB,QB,8,13,-9,-0.7,0,0,0,-1.1,0,0,0,2020
369,370,Cedrick Wilson Jr.,DAL,WR,16,3,-12,-4.0,-1,0,0,-0.8,0,0,0,2020
370,370,Adrian Killins,PHI,RB,1,1,-12,-12.0,-12,0,0,-12.0,0,0,0,2020


In [7]:
total_stats.to_csv("Rushing_2020.csv", index = False)
print("Data saved to Rushing_2020.csv")

Data saved to Rushing_2020.csv
